<a href="https://colab.research.google.com/github/steveyen127/IR_hw3/blob/main/4108056022_IR_HW3_InvertedIndex(with_timer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 114.8 MB 89 kB/s 
     |████████████████████████████████| 4.9 MB 37.7 MB/s 
     |████████████████████████████████| 5.3 MB 41.0 MB/s 
     |████████████████████████████████| 2.0 MB 55.2 MB/s 
     |████████████████████████████████| 13.5 MB 47.8 MB/s 
     |████████████████████████████████| 1.3 MB 53.6 MB/s 
     |████████████████████████████████| 1.3 MB 52.2 MB/s 
     |████████████████████████████████| 188 kB 36.2 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 163 kB 47.2 MB/s 
     |████████████████████████████████| 7.6 MB 43.3 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [6]:
from google.colab import drive
drive.mount('/drive')


Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [15]:
import json
import numpy
file_name = '/drive/MyDrive/pubmed220train.txt'

with open("documents.json", "w") as outfile:
  myList = []
  with open(file_name, "r") as f:
    text = f.readlines()
    for i in str(text).split("###"):
      doc = i.replace("\\n", "")\
            .replace("BACKGROUND\\t", "")\
            .replace("OBJECTIVE\\t", "")\
            .replace("METHODS\\t", "")\
            .replace("RESULTS\\t", "")\
            .replace("CONCLUSIONS\\t", "")
      #print(doc.split())
      new_string = " ".join(doc.split())
      id_doc = new_string.replace("',", "").replace("'","")
      id = id_doc.split()[0]
      content = " ".join(id_doc.split()[1:])
      new_content = content.replace("[", "").replace('\"', "").replace("]","")
      dictionary = {
          "id": id,
          "contents": new_content,
      }
      if id != '[':
        myList.append(dictionary)
        #print(dictionary)

  json.dump(myList, outfile)
  #print(myList)

In [18]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/samples_json \
  --index /content/indexes/documents_json \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw\
  --storeContents

2022-10-14 05:32:47,100 INFO  [main] index.IndexCollection (IndexCollection.java:248) - Setting log level to INFO
2022-10-14 05:32:47,102 INFO  [main] index.IndexCollection (IndexCollection.java:251) - Starting indexer...
2022-10-14 05:32:47,103 INFO  [main] index.IndexCollection (IndexCollection.java:252) - ============ Loading Parameters ============
2022-10-14 05:32:47,103 INFO  [main] index.IndexCollection (IndexCollection.java:253) - DocumentCollection path: /content/samples_json
2022-10-14 05:32:47,104 INFO  [main] index.IndexCollection (IndexCollection.java:254) - CollectionClass: JsonCollection
2022-10-14 05:32:47,104 INFO  [main] index.IndexCollection (IndexCollection.java:255) - Generator: DefaultLuceneDocumentGenerator
2022-10-14 05:32:47,105 INFO  [main] index.IndexCollection (IndexCollection.java:256) - Threads: 1
2022-10-14 05:32:47,105 INFO  [main] index.IndexCollection (IndexCollection.java:257) - Language: en
2022-10-14 05:32:47,105 INFO  [main] index.IndexCollection (

In [10]:
!pip install faiss-cpu --no-cache

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 5.0 MB/s 


In [21]:
from pyserini.search.lucene import LuceneSearcher
import time
start = time.perf_counter()

searcher = LuceneSearcher('/content/indexes/documents_json')
hits = searcher.search('cancer')

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
    print('{\n  "id" : "'+hits[i].docid+'",\n  "contents" : "'+hits[i].contents+'"\n}')
end = time.perf_counter()
print(f"Inverted Index Search finished in {end - start:0.4f} seconds")

 1 24747090 2.50920
{
  "id" : "24747090",
  "contents" : "We examine the role of body mass index in the assessment of prostate cancer risk . A total of 3,258 participants who underwent biopsy ( including 1,902 men with a diagnosis of prostate cancer ) were identified from the Selenium and Vitamin E Cancer Prevention Trial . The associations of body mass index with prostate cancer and high grade prostate cancer were examined using logistic regression , adjusting for age , race , body mass index adjusted prostate specific antigen , digital rectal examination , family history of prostate cancer , biopsy history , prostate specific antigen velocity , and time between study entry and the last biopsy . The prediction models were compared with our previously developed body mass index adjusted Prostate Cancer Prevention Trial prostate cancer risk calculator . Of the study subjects 49.1 % were overweight and 29.3 % were obese . After adjustment , among men without a known family history of pro